<a href="https://colab.research.google.com/github/fnko19/Deteksi-Lubang-Jalan/blob/main/TB_VISKOM_D121221054.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# OpenCV

In [ ]:
import cv2
import numpy as np

video_path = '/content/drive/MyDrive/q.mp4'
output_path = '/content/pothole_detected_with_opencv.mp4'

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Grayscale dan blur
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Hanya Canny Edge Detection
    edges = cv2.Canny(blurred, 100, 200)

    # Morphological closing
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Temukan kontur
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = cv2.contourArea(cnt)
        rect_area = w * h
        extent = area / rect_area if rect_area > 0 else 0
        aspect_ratio = w / float(h)

        # Filter sederhana berdasarkan ukuran dan bentuk
        if (area > 150 and extent > 0.4 and
            1.5 <= aspect_ratio <= 10.0):
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, "ini lubang", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Selesai! Video hasil (Canny only) disimpan di:", output_path)


✅ Selesai! Video hasil (Canny only) disimpan di: /content/pothole_detected_with_opencv.mp4


# Fungsi Manual

In [ ]:
import cv2
import numpy as np
from skimage.measure import label, regionprops

# === Gaussian Blur Manual ===
def manual_gaussian_blur(img, kernel_size=5, sigma=1.4):
    from math import exp, pi
    k = kernel_size // 2
    kernel = np.zeros((kernel_size, kernel_size), dtype=np.float32)
    for x in range(-k, k + 1):
        for y in range(-k, k + 1):
            kernel[y + k, x + k] = (1 / (2 * pi * sigma ** 2)) * exp(-(x ** 2 + y ** 2) / (2 * sigma ** 2))
    kernel /= np.sum(kernel)

    padded = np.pad(img, ((k, k), (k, k)), mode='reflect')
    output = np.zeros_like(img, dtype=np.float32)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i + kernel_size, j:j + kernel_size]
            output[i, j] = np.sum(region * kernel)

    return output.astype(np.uint8)

# === Morphology Manual ===
def dilation(img, kernel):
    k_h, k_w = kernel.shape
    pad_h, pad_w = k_h // 2, k_w // 2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    output = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i + k_h, j:j + k_w]
            if np.any(region[kernel == 1]):
                output[i, j] = 255
    return output

def erosion(img, kernel):
    k_h, k_w = kernel.shape
    pad_h, pad_w = k_h // 2, k_w // 2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    output = np.zeros_like(img)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i + k_h, j:j + k_w]
            if np.all(region[kernel == 1] == 255):
                output[i, j] = 255
    return output

def morphological_closing(img, kernel_size=(9, 9)):
    kernel = np.ones(kernel_size, dtype=np.uint8)
    dilated = dilation(img, kernel)
    closed = erosion(dilated, kernel)
    return closed

# === Canny Manual Improved ===
def manual_canny_improved(img, low_thresh=30, high_thresh=100):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.float32)

    Ix = cv2.filter2D(img, cv2.CV_64F, Kx)
    Iy = cv2.filter2D(img, cv2.CV_64F, Ky)

    G = np.hypot(Ix, Iy)
    G = G.astype(np.uint8)
    theta = np.arctan2(Iy, Ix)

    M, N = img.shape
    Z = np.zeros((M, N), dtype=np.uint8)
    angle = theta * 180. / np.pi
    angle[angle < 0] += 180

    for i in range(1, M - 1):
        for j in range(1, N - 1):
            q = 255
            r = 255
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q = G[i, j + 1]
                r = G[i, j - 1]
            elif (22.5 <= angle[i, j] < 67.5):
                q = G[i + 1, j - 1]
                r = G[i - 1, j + 1]
            elif (67.5 <= angle[i, j] < 112.5):
                q = G[i + 1, j]
                r = G[i - 1, j]
            elif (112.5 <= angle[i, j] < 157.5):
                q = G[i - 1, j - 1]
                r = G[i + 1, j + 1]
            Z[i, j] = G[i, j] if (G[i, j] >= q and G[i, j] >= r) else 0

    strong, weak = 255, 75
    res = np.zeros_like(Z, dtype=np.uint8)
    strong_i, strong_j = np.where(Z >= high_thresh)
    weak_i, weak_j = np.where((Z >= low_thresh) & (Z < high_thresh))

    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak

    for i in range(1, M - 1):
        for j in range(1, N - 1):
            if res[i, j] == weak:
                if np.any(res[i - 1:i + 2, j - 1:j + 2] == strong):
                    res[i, j] = strong
                else:
                    res[i, j] = 0

    return res

# === Video Processing ===
video_path = '/content/drive/MyDrive/q.mp4'
output_path = '/content/pothole_detected_manual.mp4'

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = manual_gaussian_blur(gray, kernel_size=5, sigma=1.4)
    edges = manual_canny_improved(blurred, 30, 100)
    closed = morphological_closing(edges, kernel_size=(9, 9))

    labeled = label(closed)
    regions = regionprops(labeled)

    for region in regions:
        if region.area < 150:
            continue

        minr, minc, maxr, maxc = region.bbox
        w, h = maxc - minc, maxr - minr
        rect_area = w * h
        extent = region.area / rect_area if rect_area > 0 else 0
        aspect_ratio = w / float(h)

        if extent > 0.4 and 1.5 <= aspect_ratio <= 10.0:
            cv2.rectangle(frame, (minc, minr), (maxc, maxr), (255, 0, 0), 2)
            cv2.putText(frame, "lubang", (minc, minr - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)

    out.write(frame)
    frame_count += 1
    if frame_count % 10 == 0:
        print(f"Memproses frame ke-{frame_count}...")

cap.release()
out.release()
print("Selesai! Video disimpan di:", output_path)


Memproses frame ke-10...
Memproses frame ke-20...
Memproses frame ke-30...
Memproses frame ke-40...
Memproses frame ke-50...
Memproses frame ke-60...
Memproses frame ke-70...
Memproses frame ke-80...
Memproses frame ke-90...
Memproses frame ke-100...
Memproses frame ke-110...
Memproses frame ke-120...
Memproses frame ke-130...
Memproses frame ke-140...
Memproses frame ke-150...
Memproses frame ke-160...
Memproses frame ke-170...
Memproses frame ke-180...
Memproses frame ke-190...
Memproses frame ke-200...
Memproses frame ke-210...
Selesai! Video disimpan di: /content/pothole_detected_manual.mp4
